<a href="https://colab.research.google.com/github/noweahc/energy/blob/main/0815%EC%A0%9C%EC%B6%9C(63%EC%A0%90_%E3%85%8B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
from tqdm import tqdm
import xgboost


In [20]:
pip install sktime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 12.8 MB/s eta 0:00:00


In [21]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sktime.forecasting.model_selection import temporal_train_test_split

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train = pd.read_csv('/content/drive/MyDrive/23energy/23data/train.csv',encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/23energy/23data/test.csv',encoding = 'utf-8')
building = pd.read_csv('/content/drive/MyDrive/23energy/23data/building_info.csv',encoding = 'utf-8')

In [6]:
train = train.rename(columns={
    '건물번호': 'num',
    '일시': 'date_time',
    '기온(C)': 'gion',
    '강수량(mm)': 'rain',
    '풍속(m/s)': 'wind',
    '습도(%)': 'hum',
    '일조(hr)': 'iljo',
    '일사(MJ/m2)': 'ilsa',
    '전력소비량(kWh)': 'power'
})
train.drop('num_date_time', axis = 1, inplace=True)

In [7]:
test = test.rename(columns={
    '건물번호': 'num',
    '일시': 'date_time',
    '기온(C)': 'gion',
    '강수량(mm)': 'rain',
    '풍속(m/s)': 'wind',
    '습도(%)': 'hum',
    '일조(hr)': 'iljo',
    '일사(MJ/m2)': 'ilsa',
    '전력소비량(kWh)': 'power'
})
test.drop('num_date_time', axis = 1, inplace=True)

In [8]:
building = building.rename(columns={
    '건물번호': 'num',
    '건물유형': 'type',
    '연면적(m2)': 'total_area',
    '냉방면적(m2)': 'cooling_area',
    '태양광용량(kW)': 'solar',
    'ESS저장용량(kWh)': 'ess',
    'PCS용량(kW)': 'pcs'
})

In [9]:
train = pd.merge(train, building, on='num', how='left')
test = pd.merge(test, building, on='num', how='left')

In [10]:
def make_time(train):
    """
    시간 관련 변수를 추가하기 위한 함수
    """
    train['date_time'] = pd.to_datetime(train.date_time)

    train['month'] = train.date_time.dt.month                    # 월(숫자)
    train['day'] = train.date_time.dt.day                        # 일(숫자)
    train['hour'] = train.date_time.dt.hour                      # 시(숫자)
    train['weekday'] = train.date_time.dt.weekday                # 요일(숫자)
    train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
    train['monthday'] = train.date_time.dt.strftime('%m%d')      # 월일

    return train

In [11]:
train = make_time(train)
test = make_time(test)

In [12]:
# 문자열을 실수형으로 변환하고 "-"을 0으로 대체합니다.
train["solar"] = train["solar"].apply(lambda x: float(x) if x != "-" else 0)
train["ess"] = train["ess"].apply(lambda x: float(x) if x != "-" else 0)
train["pcs"] = train["pcs"].apply(lambda x: float(x) if x != "-" else 0)

# 문자열을 실수형으로 변환하고 "-"을 0으로 대체합니다.
test["solar"] = test["solar"].apply(lambda x: float(x) if x != "-" else 0)
test["ess"] = test["ess"].apply(lambda x: float(x) if x != "-" else 0)
test["pcs"] = test["pcs"].apply(lambda x: float(x) if x != "-" else 0)

In [13]:
#토요일, 일요일은 주말로 처리
#전국동시지방선거일, 현충일, 광복절도 주말로 처리

train.loc[train.weekday >= 5, 'holiday'] = 1
train.loc[(train.weekday < 5), 'holiday'] = 0

#train.loc[train.monthday == '0601', 'holiday'] = 1
#train.loc[train.monthday == '0606', 'holiday'] = 1
#train.loc[train.monthday == '0815', 'holiday'] = 1
train.loc[train['monthday'].isin(['0601', '0606', '0815']), 'holiday'] = 1

test.loc[test.weekday >= 5, 'holiday'] = 1
test.loc[test.weekday < 5, 'holiday'] = 0

In [14]:
#건물유형 숫자로 처리하는 라벨인코딩

label_mapping = {
    '건물기타': 0,
    '공공': 1,
    '대학교': 2,
    '데이터센터': 3,
    '백화점및아울렛': 4,
    '병원': 5,
    '상용': 6,
    '아파트': 7,
    '연구소': 8,
    '지식산업센터': 9,
    '할인마트': 10,
    '호텔및리조트': 11
}

train['type'] = train['type'].replace(label_mapping)
test['type'] = test['type'].replace(label_mapping)

In [15]:
#파생변수 : 체감온도
train['sensory_temp'] = 13.12 + 0.6215 * train['gion'] - 11.37 * (train['wind'] ** 0.16) + 0.3965 * (train['wind'] ** 0.16) * train['gion']
test['sensory_temp'] = 13.12 + 0.6215 * test['gion'] - 11.37 * (test['wind'] ** 0.16) + 0.3965 * (test['wind'] ** 0.16) * test['gion']

In [16]:
#https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
#아이디어 참고
#시간의 순환적 성격 반영하기

train['sin_time'] = np.sin(2*np.pi*train.hour/24)
train['cos_time'] = np.cos(2*np.pi*train.hour/24)

test['sin_time'] = np.sin(2*np.pi*test.hour/24)
test['cos_time'] = np.cos(2*np.pi*test.hour/24)

In [17]:
#https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent
#아이디어 참고
#기온과 습도를 고려한 불쾌지수(THI)
#CDH(Cooling Degree Hour) : 기온이 실제 냉방 가동에 이르기까지의 시간적 오차를 고려

train['THI'] = 9/5*train['gion'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32
test['THI'] = 9/5*test['gion'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['gion'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['num'] == num]
    cdh = CDH(temp['gion'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

In [23]:
#성능 평가 지표 : weighted_mse
#### alpha를 argument로 받는 함수로 실제 objective function을 wrapping하여 alpha값을 쉽게 조정할 수 있도록 작성했습니다.
# custom objective function for forcing model not to underestimate
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

In [24]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100

In [26]:
train.drop(['date_time','rain','iljo','ilsa','solar','ess','pcs','hour','monthday'], axis = 1, inplace = True)
test.drop(['date_time','rain','solar','ess','pcs','hour','monthday'], axis = 1, inplace = True)

In [36]:
#best iteration 찾기 : 하이퍼파라미터 후보 df인 xgb_params에 저장하면 됨
scores = []   # smape 값을 저장할 list
best_it = []  # best interation을 저장할 list
for i in tqdm(range(100)):
    y = train.loc[train.num== i+1, 'power']
    x = train.loc[train.num == i+1, ].iloc[:, 3:]
    y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)

    xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

    xgb_model.set_params(**{'objective':weighted_mse(100)}) # alpha = 100으로 고정

    xgb_model.fit(x_train, y_train, eval_set=[(x_train, y_train),
                                            (x_valid, y_valid)], early_stopping_rounds=300, verbose=False)
    y_pred = xgb_model.predict(x_valid)
    pred = pd.Series(y_pred)

    sm = SMAPE(y_valid, y_pred)
    scores.append(sm)
    best_it.append(xgb_model.best_iteration) ## 실제 best iteration은 이 값에 +1 해주어야 함.

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
100%|██████████| 100/100 [01:16<00:00,  1.30it/s]


In [37]:
#weighted MSE의 alpha값 튜닝단계 : 하이퍼파라미터 후보 df인 xgb_params에 저장하면됨
alpha_list = []
smape_list = []
for i in tqdm(range(100)):
    y = train.loc[train.num == i+1, 'power']
    x = train.loc[train.num == i+1, ].iloc[:, 3:]
    y_train, y_test, x_train, x_test = temporal_train_test_split(y = y, X = x, test_size = 168)
    xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)

    xgb_model.fit(x_train, y_train)
    pred0 = xgb_model.predict(x_test)
    best_alpha = 0
    score0 = SMAPE(y_test,pred0)

    for j in [1, 3, 5, 7, 10, 25, 50, 75, 100]:
        xgb_model = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
        xgb_model.set_params(**{'objective' : weighted_mse(j)})

        xgb_model.fit(x_train, y_train)
        pred1 = xgb_model.predict(x_test)
        score1 = SMAPE(y_test, pred1)
        if score1 < score0:
            best_alpha = j
            score0 = score1

    alpha_list.append(best_alpha)
    smape_list.append(score0)
    print("building {} || best score : {} || alpha : {}".format(i+1, score0, best_alpha))

  1%|          | 1/100 [00:08<14:13,  8.62s/it]

building 1 || best score : 0.024593073735724086 || alpha : 100


  2%|▏         | 2/100 [00:13<10:22,  6.35s/it]

building 2 || best score : 0.025594065433553255 || alpha : 100


  3%|▎         | 3/100 [00:18<09:07,  5.65s/it]

...

building 98 || best score : 0.03578518664358602 || alpha : 100


 99%|█████████▉| 99/100 [11:41<00:08,  8.79s/it]

building 99 || best score : 0.024052575725348888 || alpha : 75


100%|██████████| 100/100 [11:52<00:00,  7.13s/it]

building 100 || best score : 0.027790464102279727 || alpha : 100


In [48]:
#seed별로 예측값이 바뀌니까 seed 0부터 6까지 평균내서
preds = np.array([])
for i in tqdm(range(100)):

    pred_df = pd.DataFrame()   # 시드별 예측값을 담을 data frame

    for seed in [0,1,2,3,4,5]: # 각 시드별 예측
        y_train = train.loc[train.num == i+1, 'power']
        x_train, x_test = train.loc[train.num == i+1, ].iloc[:, 3:], test.loc[test.num == i+1, ].iloc[:,1:]
        #x_test['power'] = 0 ####여기 이상 ..
        x_test = x_test[x_train.columns]
        '''
        xgb = XGBRegressor(seed = seed, n_estimators = best_it[i], eta = 0.01,
                           min_child_weight = xgb_params.iloc[i, 2], max_depth = xgb_params.iloc[i, 3],
                           colsample_bytree=xgb_params.iloc[i, 4], subsample=xgb_params.iloc[i, 5])
                           '''
        xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
        '''
        if xgb_params.iloc[i,6] != 0:  # 만약 alpha가 0이 아니면 weighted_mse 사용
            xgb.set_params(**{'objective':weighted_mse(xgb_params.iloc[i,6])})
        '''
        xgb.fit(x_train, y_train)
        y_pred = xgb.predict(x_test)
        pred_df.loc[:,seed] = y_pred   # 각 시드별 예측 담기

    pred = pred_df.mean(axis=1)        # (i+1)번째 건물의 예측 =  (i+1)번째 건물의 각 시드별 예측 평균값
    preds = np.append(preds, pred)

100%|██████████| 100/100 [04:35<00:00,  2.76s/it]


In [49]:
submission = pd.read_csv('/content/drive/MyDrive/23energy/23data/sample_submission.csv')
submission['answer'] = preds
submission.to_csv('submission_xgb_0815.csv', index = False)

In [50]:
y_train = train.loc[train.num == 1, 'power']
x_train, x_test = train.loc[train.num == i+1, ].iloc[:, 3:], test.loc[test.num == i+1, ].iloc[:,1:]

In [51]:
train

,num,gion,wind,hum,power,type,total_area,cooling_area,month,day,weekday,dayofyear,holiday,sensory_temp,sin_time,cos_time,THI,CDH
0,1,18.6,0.9,42.0,1085.28,0,110634.00,39570.00,6,1,2,152,1.0,20.751584,0.000000,1.000000,49.6576,-7.4
1,1,18.0,1.1,45.0,1047.36,0,110634.00,39570.00,6,1,2,152,1.0,20.008954,0.258819,0.965926,47.7625,-15.4
2,1,17.7,1.5,45.0,974.88,0,110634.00,39570.00,6,1,2,152,1.0,19.476910,0.500000,0.866025,47.2225,-23.7
3,1,16.7,1.4,48.0,953.76,0,110634.00,39570.00,6,1,2,152,1.0,18.487958,0.707107,0.707107,44.7856,-33.0
4,1,18.4,2.8,43.0,986.40,0,110634.00,39570.00,6,1,2,152,1.0,19.751534,0.866025,0.500000,49.0061,-40.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100,23.1,0.9,86.0,881.04,11,57497.84,40035.23,8,24,2,236,0.0,25.302757,-0.965926,0.258819,63.6624,-19.6
203996,100,22.4,1.3,86.0,798.96,11,57497.84,40035.23,8,24,2,236,0.0,24.446518,-0.866025,0.500000,62.4024,-20.2
203997,100,21.3,1.0,92.0,825.12,11,57497.84,40035.23,8,24,2,236,0.0,23.433400,-0.707107,0.707107,64.1976,-22.3
203998,100,21.0,0.3,94.0,640.08,11,57497.84,40035.23,8,24,2,236,0.0,23.661275,-0.500000,0.866025,65.0744,-25.1


In [52]:
test

,num,gion,wind,hum,type,total_area,cooling_area,month,day,weekday,dayofyear,holiday,sensory_temp,sin_time,cos_time,THI,CDH
0,1,23.5,2.2,72,0,110634.00,39570.00,8,25,3,237,0.0,25.397063,0.000000,1.000000,58.3456,-2.5
1,1,23.0,0.9,72,0,110634.00,39570.00,8,25,3,237,0.0,25.201620,0.258819,0.965926,57.4456,-5.5
2,1,22.7,1.5,75,0,110634.00,39570.00,8,25,3,237,0.0,24.699788,0.500000,0.866025,57.8725,-8.8
3,1,22.1,1.3,78,0,110634.00,39570.00,8,25,3,237,0.0,24.136018,0.707107,0.707107,57.9376,-12.7
4,1,21.8,1.0,77,0,110634.00,39570.00,8,25,3,237,0.0,23.942400,0.866025,0.500000,56.9961,-16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,22.5,0.9,84,11,57497.84,40035.23,8,31,2,243,0.0,24.695934,-0.965926,0.258819,61.4824,-34.5
16796,100,20.7,0.4,95,11,57497.84,40035.23,8,31,2,243,0.0,23.253851,-0.866025,0.500000,65.2725,-34.4
16797,100,20.2,0.4,98,11,57497.84,40035.23,8,31,2,243,0.0,22.771886,-0.707107,0.707107,66.7056,-35.3
16798,100,20.1,1.1,97,11,57497.84,40035.23,8,31,2,243,0.0,22.159549,-0.500000,0.866025,65.7281,-36.8
